Reference:

https://mpatacchiola.github.io/blog/2016/12/09/dissecting-reinforcement-learning.html
https://github.com/mpatacchiola/dissecting-reinforcement-learning/tree/master/src/1

In [1]:
import numpy as np

In [3]:
# transition matrix loaded from file
# T[s, a, s'] = probability of transition from start state s, take action a, and enter state s'
# for this issue, there are 3*4 = 12 states, 4 actions (left, right, up, down)
T = np.load("resources\T.npy")

In [7]:
print(T.shape)
print(T[0, 0, 0])

(12, 12, 4)
0.9


In [10]:
T_tran = np.transpose(T, (0, 2, 1))
print(T_tran.shape)

(12, 4, 12)


In [11]:
u = np.array([0.812, 0.868, 0.918,   1.0,
                   0.762,   0.0, 0.660,  -1.0,
                   0.705, 0.655, 0.611, 0.388])

In [12]:
r = np.array([-0.04, -0.04, -0.04,   -0.04,
                   -0.04,   -0.04, -0.04, -0.04,
                   -0.04, -0.04, -0.04, 1])

In [21]:
print("u.shape: ", u.shape, "r.shape: ", r.shape)

u.shape:  (12,) r.shape:  (12,)


In [22]:
u_temp = T_tran.dot(u)
print(u_temp.shape)

(12, 4)


In [23]:
u_max = np.max(u_temp, axis=1)
print(u_max.shape)

(12,)


In [25]:
u_new = r + 1.0 * u_max
u_shaped = u_new.reshape((3, 4))
print(u_shaped)

[[ 0.8118  0.868   0.9178 -0.04  ]
 [ 0.762  -0.04    0.6604 -0.04  ]
 [ 0.7056  0.655   0.6111  1.4276]]


## Value Iteration

In [30]:
# action: 0=Up, 1=Left, 2=Down, 3=Right
def state_value(current_value, T, reward, gamma):
    """
    It's used in value iteration algorithm
    params:
        current_value: current value of each state. shape: (state_num, )
        T: transition matrix. shape: (state_num, state_num, action_num)
        reward: reward in each state. shape: (state_num, )
        gamma: discount factor. 0 <= gamma <=1.0
    
    return:
        new value of each state. shape: (state_num, ).
        the policy. shape: (state_num, )
    """
    
    T_tran = np.transpose(T, (0, 2, 1)) # T_tran shape: (state_num, action_num, state_num)
    expected_value = T_tran.dot(current_value) # shape: (state_num, action_num)
    new_value = reward + gamma * np.max(expected_value, axis = 1)
    policy = np.argmax(expected_value, axis = 1)
    return (new_value, policy)

In [27]:
reward = np.array([-0.04, -0.04, -0.04,  +1.0,
                   -0.04,   0.0, -0.04,  -1.0,
                   -0.04, -0.04, -0.04, -0.04])

In [46]:
def print_policy(policy, shape):
    """
    param:
        policy: action: 0=Up, 1=Left, 2=Down, 3=Right, -1=terminal, -2=obstacle. Shape: (state_num, )
        shape: tuple. (row, column)
    return: null
    """
    
    def action_translate(index):
        if index == -1: return " * "
        elif index == 0: return " ^ "
        elif index == 1: return " < "
        elif index == 2: return " v "
        elif index == 3: return " > "
        elif index == -2: return " # "
        else: return " - "
        
    v_action_translate = np.vectorize(action_translate)
    symbols = v_action_translate(policy)
    print(symbols.reshape(shape))

In [51]:
state_num = 12
gamma = 0.999 # different gamma, the optimal policy and value is different
iteration = 0
epsilon = 0.01 # stopping criteria small value

value_history = list()
policy_history = list()
value = np.zeros((state_num, ))
for i in range(100):
    delta = 0.0
    iteration += 1
    
    new_value, new_policy = state_value(value, T, reward, gamma)
    value_history.append(new_value)
    policy_history.append(new_policy)
    
    delta = np.max(np.abs(new_value - value))
    value = new_value

    if delta < epsilon * (1-gamma) / gamma:
        print("Iteration: %d, delta=%f" % (iteration, delta))
        print(value.reshape(3, 4))
        print("Policy: ")
        new_policy[5] = -2
        new_policy[3] = new_policy[7] = -1
        print_policy(new_policy, (3, 4))
        break;

Iteration: 26, delta=0.000010
[[ 0.80796343  0.86539911  0.91653199  1.        ]
 [ 0.75696619  0.          0.65836281 -1.        ]
 [ 0.69968237  0.64881928  0.60471582  0.38149581]]
Policy: 
[[' > ' ' > ' ' > ' ' * ']
 [' ^ ' ' # ' ' ^ ' ' * ']
 [' ^ ' ' < ' ' < ' ' < ']]


## Policy iteration

In [ ]:
def iterate_policy(current_policy, current_value, T, reward, gamma):
    """
    return:
        new value and new policy. shape: (state_num, )
    """